# Bug Reference Utils

Code snippets useful for working with bug references and bug trackers

In [ ]:
include("../src/init.jl") # Configure and install (if necessary) the JDP project

import JDP.BugRefs
import JDP.Trackers.Bugzilla

Extract bug references from the specified string and display them as hyper links using Markdown.

In [12]:
BugRefs.extract_refs("bsc#1111458 bsc#1111464 bsc#1111466 bsc#1111469") |> BugRefs.to_md

[bsc#1111458](https://bugzilla.suse.com/show_bug.cgi?id=1111458)
[bsc#1111464](https://bugzilla.suse.com/show_bug.cgi?id=1111464)
[bsc#1111466](https://bugzilla.suse.com/show_bug.cgi?id=1111466)
[bsc#1111469](https://bugzilla.suse.com/show_bug.cgi?id=1111469)



In [7]:
BugRefs.extract_refs("bsc#1110703 bsc#1111488") |> BugRefs.to_md
    

[bsc#1110703](https://bugzilla.suse.com/show_bug.cgi?id=1110703)
[bsc#1111488](https://bugzilla.suse.com/show_bug.cgi?id=1111488)



Displaying them as hyperlinks is OK, but we can go further by automatically fetching data from Bugzilla and displaying it here. First we need to login to Bugzilla, if you have not set a user name and password in the trackers config file then you will be prompted for them.

> WARNING: This initially uses HTTP basic auth which inserts your user name and password into the URI, however subsequent requests use a cookie stored in the ses object

In [14]:
ses = Bugzilla.login("bsc");

User Name: rpalethorpe
Password: ········


Now we can get a bug. The object returned is a Dictionary derived from Bugzilla's XML output.

In [15]:
bug = Bugzilla.get_bug(ses, 1103543)

Dict{Union{String, Symbol},Any} with 41 entries:
  "rep_platform"           => "Other"
  "bug_severity"           => "Normal"
  "cclist_accessible"      => "1"
  "bug_status"             => "RESOLVED"
  "cf_nts_priority"        => SubString{String}["", ""]
  "classification_id"      => "27"
  "reporter_accessible"    => "1"
  "bug_id"                 => "1103543"
  "keywords"               => ""
  "cf_marketing_qa_status" => "---"
  "cf_it_deployment"       => "---"
  "estimated_time"         => "0.00"
  "classification"         => "SUSE Linux Enterprise Server"
  "priority"               => "P2 - High"
  "short_desc"             => "xftests generic/502 fails for btrfs"
  "cf_foundby"             => SubString{String}["---", "---"]
  "creation_ts"            => "2018-08-02 09:45:00 +0000"
  "assigned_to"            => EzXML.Node(<ELEMENT_NODE@0x00000000059f2f10>)
  "op_sys"                 => "Other"
  "version"                => "Beta 2"
  "qa_contact"             => EzXML.Node(<ELEMEN

This can be converted to a Markdown object which Jupyter will display nicely.

In [16]:
md = Bugzilla.to_md(bug)

**P2 - High**(*Normal*) RESOLVED: xftests generic/502 fails for btrfs


We can take a bunch of bugs from the report notebook and display a summary of all of them.

In [6]:
bugs = []

for ref in ("bsc#1099134", "bsc#1074293", "poo#35347", "bsc#1099173", "bsc#1102358", "bsc#1102250", "bsc#1108010", "bsc#1108028", "poo#40424")
    if startswith(ref, "bsc")
        push!(bugs, Bugzilla.get_bug(ses, parse(Int, ref[5:end])))
    end
end

bugs

7-element Array{Any,1}:
 Dict{Union{String, Symbol},Any}("rep_platform"=>"PowerPC-64","bug_severity"=>"Normal","cclist_accessible"=>"1","bug_status"=>"NEW","dependson"=>"1098652","cf_nts_priority"=>SubString{String}["", ""],"classification_id"=>"27","reporter_accessible"=>"1","bug_id"=>"1099134","keywords"=>""…)            
 Dict{Union{String, Symbol},Any}("rep_platform"=>"Other","bug_severity"=>"Major","cclist_accessible"=>"1","bug_status"=>"RESOLVED","cf_nts_priority"=>SubString{String}["", ""],"classification_id"=>"2","reporter_accessible"=>"1","bug_id"=>"1074293","keywords"=>"","cf_marketing_qa_status"=>"---"…)     
 Dict{Union{String, Symbol},Any}("rep_platform"=>"Other","bug_severity"=>"Normal","cclist_accessible"=>"1","bug_status"=>"RESOLVED","cf_nts_priority"=>SubString{String}["", ""],"classification_id"=>"27","reporter_accessible"=>"1","bug_id"=>"1099173","keywords"=>"","cf_marketing_qa_status"=>"---"…)   
 Dict{Union{String, Symbol},Any}("rep_platform"=>"Other","bug_severity

In [11]:
using Markdown
import Markdown: Paragraph, MD

map(Bugzilla.to_md, bugs) |> MD

**P3 - Medium**(*Normal*) NEW: Btrfs fallocate PUNCH_HOLE | KEEP_SIZE fails on filled up FS on ppc64le

**P3 - Medium**(*Major*) RESOLVED: VUL-0: CVE-2018-1000001: glibc: privilege escalation bug in glibc

**P2 - High**(*Normal*) RESOLVED: inotify overlayfs fixes

**P5 - None**(*Major*) RESOLVED: mm/pgtable-generic.c:39: bad pmd and hvc hvc0: unknown flag

**P1 - Urgent**(*Critical*) RESOLVED: cgroup related NULL pointer dereference during shutdown

**P2 - High**(*Normal*) RESOLVED: MADV_FREE does not seem to work on aarch64

**P2 - High**(*Normal*) RESOLVED: statx() does not seem to fill in stx_attributes on Btrfs
